In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import datetime
import tools
import random

In [ ]:
filename = 'experiments.log' # Normal local tests
#filename = 'experiments_E.log' # manual virtual wall tests
filename = 'experiments_0.log' # Virtual wall tests

interval = 19
batch_count = 4
experiment_batches = [f'experiments_{i * interval}.log' for i in range(batch_count)]
print(experiment_batches)

selected_batch = 1
filename = experiment_batches[selected_batch-1]
print(filename)

filename = 'experiments_local.log' # Normal local tests
filename = 'experiments_local2.log' # Normal local tests
filename = 'experiments_local3.log' # Normal local tests


In [ ]:
experiments = tools.get_all_experiments(filename, align=True, interpolate=False, resample=False)
experiments_resampled = tools.get_all_experiments(filename, align=True, interpolate=False, resample=True)
# [experiment['directory'] for experiment in experiments]
#len(experiments)

In [ ]:
remove_old = False
if remove_old:
    # Search for the lowest experiment id, and remember the dir that belongs to it
    lowest_id = 1000000000000
    lowest_dir = 0
    for experiment in experiments:
        if int(experiment['id']) < lowest_id:
            lowest_id = int(experiment['id'])
            lowest_dir = int(experiment['directory'])

    # dir is an integer as a string, we want to filter out all the experiments that have a lower dir
    experiments = [experiment for experiment in experiments if int(experiment['directory']) >= lowest_dir]

    len(experiments)

    # dir is an integer as a string, we want to filter out all the experiments that have a lower dir
    experiments_resampled = [experiment_resampled for experiment_resampled in experiments_resampled if int(experiment_resampled['directory']) >= lowest_dir]

len(experiments)

#experiments[0]['dfs']['server_http']

In [ ]:
ii = [ exp['info'] for exp in experiments]
[jj['dir'] + ' ' + jj['fec'] + ' ' + jj['seg_dur'] for jj in ii]

In [ ]:
experiments[0]['info']

In [ ]:
def reset_id_index(experiments):
    if len(experiments) == 0:
        return
    # Search for the lowest id in the array: el['id]
    min_id = int(experiments[0]['id'])
    for experiment in experiments:
        current_id = int(experiment['id'])
        if current_id < min_id:
            min_id = current_id
    # Decrease the id of all the experiments by min_id
    for experiment in experiments:
        new_id = str(int(experiment['id']) - min_id)
        experiment['id'] = new_id
        experiment['info']['id'] = new_id

reset_id_index(experiments)

In [ ]:
def take_window(df, column):
    # Remove all the rows with 0 values, this removes the first rows
    df = df.dropna().loc[(df!=0).any(axis=1)]
    if (len(df) == 0):
        return df
    # Take the difference between rows, we want to find the last part where data is not increasing
    df_diff = df.diff()
    # Remove all non increasing rows
    df_diff = df_diff.loc[(df_diff!=0).any(axis=1)]
    if (len(df) == 0):
        return df
    # Get the last index of df_diff
    last_index = df_diff.iloc[-1:].index[0]
    # remove all rows from df after last_index
    return df.loc[:last_index]


def get_values(experiments, event, should_round=False):
    dfs_found = {}
    for experiment in experiments:
        df = tools.get_metrics_by_events(
            experiment['dfs'],
            [event],
            interpolate=False,
        )
        if df.size > 0:
            df.columns = [experiment['id'] + ' ' + col for col in df.columns]
            df_window = take_window(df, event)
            if (len(df_window) == 0):
                continue
            # Get the first index of df_window
            first_index = df_window.index[0]
            # Get the last index of df_window
            last_index = df_window.iloc[-1:].index[0]
            # Get the first value
            first_value = df_window.iloc[0].item()
            # Get the last value
            last_value = df_window.iloc[-1].item()
            # Get the sum of all values
            sum_value = df_window.sum().item()
            # Get the max value
            max_value = df_window.max().item()
            # Get the difference between the first and last value
            value_diff = (df_window.iloc[-1] - df_window.iloc[0]).item()
            # Get the average value
            avg_value = sum_value / len(df_window)

            result = {
                'id': experiment['id'],
                'event': event,
                'first': round(first_value, 3) if should_round else first_value,
                'last': round(last_value, 3) if should_round else last_value,
                'sum': round(sum_value, 3) if should_round else sum_value,
                'max': round(max_value, 3) if should_round else max_value,
                'diff': round(value_diff, 3) if should_round else value_diff,
                'avg': round(avg_value, 3) if should_round else avg_value,
                'start_index': first_index,
                'end_index': last_index,
            }
            dfs_found[experiment['id']] = result

    return dfs_found

def convert_to_df(dictionaries, column_name, value_name='value'):
    df = pd.DataFrame(dictionaries).T
    if (len(df) == 0):
        return df
    df = df.pivot(index='id', columns='event', values=value_name).reset_index()
    df = df.rename_axis(None, axis=1)
    df = df.fillna(0)  # Replace NaN values with 0 if necessary
    df['id'] = df['id'].astype(int)
    df.set_index('id', inplace=True)  # Set 'id' as the index
    # Get the event name from dict
    event = list(dictionaries.values())[0]['event']

    df = df.rename(columns={event: column_name})
    return df.sort_index(ascending=True)

In [ ]:
# Get two datetimes 1970 00:00:20 and 00:01:20

def get_first_from_nested_list(nested_list):
    if '[' in nested_list:
        # Remove the '[' and ']'
        nested_list = nested_list.replace('[', '').replace(']', '')
        # Split by ','
        nested_list = nested_list.split(',')[0]
        # Remove the spaces
        nested_list = nested_list.strip()
        if '[' in nested_list:
            # Remove the '[' and ']'
            nested_list = nested_list.replace('[', '').replace(']', '')
            # Split by ','
            nested_list = nested_list.split(',')[0]
            # Remove the spaces
            nested_list = nested_list.strip()

    # Remove ' and " from the beginning and end
    nested_list = nested_list.strip('\'"')
    return nested_list

def get_clients_sleep_time(clients_sleep_time):
    clients_sleep_time = get_first_from_nested_list(clients_sleep_time)
    return float(clients_sleep_time)


clients_sleep_time = str(experiments[0]['info']['clients_sleep_time'])
clients_sleep_time = get_clients_sleep_time(clients_sleep_time)
segment_duration = int(experiments[0]['info']['seg_dur'])


offset_time_delta = datetime.timedelta(seconds=clients_sleep_time - segment_duration)


start_time = datetime.datetime(1970, 1, 1, 0, 0)
end_time = datetime.datetime(1970, 1, 1, 0, 1, 36)

avg_start_time = datetime.datetime(1970, 1, 1, 0, 0, 30)
avg_end_time = datetime.datetime(1970, 1, 1, 0, 1, 15)

print(offset_time_delta)
print(start_time)
print(end_time)

In [ ]:
legend_map = {
    'live_latency': 'Live latency',
}

def get_legend_event(event):
    if event in legend_map:
        return legend_map[event]
    
    # Replace all underscores with spaces
    event = event.replace('_', ' ')
    # Capitalize the first letter of the event
    event = event.capitalize()
    return event

In [ ]:
def video_title_map(video, seg_dur):
    if video == 'avatarcmaf8' or video == 'avatarcmaf8_1':
        return 'CMAF-CTE'
    if video == 'avatarcmaf1' or video == 'avatarcmaf1_1':
        return f'{seg_dur} sec segments'

    return video



In [ ]:
def plot_events(experiments,
                events,
                start_time=None,
                end_time=None,
                title='',
                legend_title=None,
                xlabel='Livestream time',
                ylabel='',
                figsize=(10, 3),
                linewidth=2,
                legend=True,
                mc_only=False, # Only show results with multicast
                no_mc_only=False, # Only show results without multicast
                fec_only=False, # Only show results with FEC
                no_fec_only=False, # Only show results without FEC
                show_fec_legend=False,
                legend_location='upper right',
                legend_columns=1,
                losses_to_plot=None,
                offset_time_delta=offset_time_delta,
                offset_time_on_playback=True,
                offset_time_on_mc=True,
                avg_between=[avg_start_time, avg_end_time],
                show_video_title=True,
                show_experiment_dir=False,
                show_loss=False,
                grid=False,
                colors=None,
                bottom_limit=None,
                top_limit=None,
                correct_latency=False,
                extra_event_filter=None,
                data_modifier=None):
    combined_data = []
    averages = {}
    # Check if xlabel ends with ')', if not add ' (s)'
    if xlabel[-1] != ')':
        xlabel += ' (s)'

    for experiment in experiments:
        # Get the amount of loss for this experiment
        loss = experiment['info']['loss']
        if losses_to_plot is not None and float(loss) not in [float(l) for l in losses_to_plot]:
            continue

        multicast_enabled = experiment['info']['disable_multicast'] == '0' or experiment['info']['disable_multicast'] == 'False'
        fec_enabled = experiment['info']['fec'] == '1'
        video = get_first_from_nested_list(experiment['info']['videos'])
        seg_dur = int(experiment['info']['seg_dur'])
        video_title = video_title_map(video, seg_dur) + ' ' if show_video_title else ''
        loss_text = (f'{loss} % loss' if multicast_enabled else 'No MC') if show_loss else ''
        clients_sleep_time = get_clients_sleep_time(experiment['info']['clients_sleep_time'])
        multicast_sleep_time = get_clients_sleep_time(experiment['info']['multicast_sender_sleep_time'])
        wait_time = clients_sleep_time - multicast_sleep_time

        experiment_title = video_title + loss_text + (' (FEC)' if fec_enabled else '')

        if show_experiment_dir:
            experiment_title += f' ({experiment["info"]["dir"]})'

        if mc_only and not multicast_enabled:
            continue
        if no_mc_only and multicast_enabled:
            continue

        if fec_only and not fec_enabled and multicast_enabled:
            continue
        if no_fec_only and fec_enabled and multicast_enabled:
            continue

        if offset_time_on_playback:
            playback_data = tools.get_metrics_by_events(experiment['dfs'], ['playing'], interpolate=True)
            # Get the first index where a value in the rows is not 0
            # Replace NaN values with 0
            playback_data = playback_data.fillna(0)
            playback_data = playback_data.loc[(playback_data!=0).any(axis=1)]
            # Get the first index of playback_data
            if len(playback_data) > 0 :
                offset_time_delta = playback_data.index[0]
                if start_time is not None:
                    # Substract the start_time from the offset_time_delta
                    offset_time_delta = offset_time_delta - start_time


        if offset_time_on_mc and multicast_enabled:
            if offset_time_on_playback:
                offset_time_delta = offset_time_delta - datetime.timedelta(seconds=wait_time)
            else:
                # Get the first index of the multicast data
                multicast_data = tools.get_metrics_by_events(experiment['dfs'], ['is_multicasting'], interpolate=True)
                # Get the first index of multicast_data
                if len(multicast_data) > 0 :
                    offset_time_delta = multicast_data.index[0]
                    print(f'Offset time delta: {offset_time_delta}')
                    if start_time is not None:
                        # Substract the start_time from the offset_time_delta
                        offset_time_delta = offset_time_delta - start_time
                    if end_time is not None:
                        # Add the segment duration to the offset_time_delta
                        offset_time_delta = offset_time_delta + datetime.timedelta(seconds=segment_duration)
            


        experiment_data = tools.get_metrics_by_events(experiment['dfs'], events, interpolate=True)
        # Check if the experiment has any data
        if experiment_data.size > 0:
            # The index is a datetime, we want to drop all the values before the offset_time
            # experiment_data = experiment_data.loc[experiment_data.index >= offset_time_delta]
            # Subtract the offset_time from the index
            experiment_data.index = experiment_data.index - offset_time_delta

            if extra_event_filter is not None:
                # Only keep the columns where extra_event_filter(column) is true
                experiment_data = experiment_data.loc[:, experiment_data.columns.map(extra_event_filter)]

            if data_modifier is not None:
                # check if it is a list
                if isinstance(data_modifier, list):
                    for modifier in data_modifier:
                        experiment_data = modifier(experiment_data)
                else:
                    experiment_data = data_modifier(experiment_data)

            if correct_latency:
                # Correct the latency
                experiment_data = experiment_data + wait_time

            # Rename the columns
            experiment_data.columns = [experiment_title for col in experiment_data.columns]

            # Take a snapshot between avg_start_time and avg_end_time
            avg_between = experiment_data.loc[avg_start_time:avg_end_time]
            # Calculate the avg per column
            avg_between = avg_between.mean()
            # Print the avgs one by one
            for index, value in avg_between.items():
                print(f'Avg {index}: {value}')
                averages[index] = value

            combined_data.append(experiment_data)

    # From averages, get all the keys with (FEC) in them
    fec_keys = [key for key in averages.keys() if '(FEC)' in key]
    # From averages, get all the keys without (FEC) in them and without 'No MC'
    no_fec_keys = [key for key in averages.keys() if '(FEC)' not in key and 'No MC' not in key]
    # Get the average of the averages
    if len(fec_keys) > 0:
        avg_fec = sum([averages[key] for key in fec_keys]) / len(fec_keys)
        print(f'Avg FEC: {avg_fec}')
    if len(no_fec_keys) > 0:
        avg_no_fec = sum([averages[key] for key in no_fec_keys]) / len(no_fec_keys)
        print(f'Avg No FEC: {avg_no_fec}')


    tools.plot_data(combined_data, start_time=start_time, end_time=end_time, title=title, legend_title=legend_title, xlabel=xlabel, ylabel=ylabel, figsize=figsize, linewidth=linewidth, grid=grid, legend=legend, legend_location=legend_location, legend_columns=legend_columns, bottom_limit=bottom_limit, top_limit=top_limit, colors=colors, show_fec_legend=show_fec_legend) 

In [ ]:
def proxies_event_filter(event):
    return 'proxy' in event

def server_event_filter(event):
    return 'server' in event

def client_event_filter(event):
    return 'lient' in event

In [ ]:
def divide_data_by_1k(df):
    return df / 1000

def multiply_data_by_1k(df):
    return df * 1000

def remove_columns_with_only_zeros(df):
    # Set the threshold for considering values as "zero"
    threshold = 1.0e-10  # Adjust this threshold as needed

    # Remove columns with values close to zero
    return df.loc[:, (np.abs(df) > threshold).any(axis=0)]

def fill_nan_with_zeros(df):
    return df.fillna(0)

def fill_nan_with_forward(df):
    # Fill NaN values with the previous value
    return df.ffill()

def fill_nan_with_backward(df):
    # Fill NaN values with the next value
    return df.bfill()

def fill_nan_with_linear(df):
    # Fill NaN values with a linear interpolation
    return df.interpolate(method='linear')

def remove_nan(df):
    return df.dropna()

def remove_ffill(df):
    # Set all the values that are within threshold of the previous value to NaN
    threshold = 1.0e-10  # Adjust this threshold as needed
    return df.mask(df.sub(df.shift()).abs().lt(threshold))

def remove_ffill_big(df):
    df_copy = df.copy()
    threshold = 0.002

    # Round everything to 0.001
    df_copy = df_copy.round(3)

    for col in df_copy.columns:
        diff_values = df_copy[col].diff()
        indexes_to_replace = diff_values.abs() <= threshold

        # Replace the values with NaN
        df_copy.loc[indexes_to_replace, col] = np.nan

    return df_copy

def set_nan_if_next_is_different(df):
    df_copy = df.copy()
    for col in df_copy.columns:
        diff_values = df_copy[col].diff()
        indexes_to_replace = diff_values != 0

        # Replace the values with NaN
        df_copy.loc[indexes_to_replace, col] = np.nan

    return df_copy

def set_nan_if_next_is_bigger(df):
    df_copy = df.copy()
    for col in df_copy.columns:
        diff_values = df_copy[col].diff()
        indexes_to_replace = diff_values > 0

        # Replace the values with NaN
        df_copy.loc[indexes_to_replace, col] = np.nan

    return df_copy

def round_down(df):
    return df.round(0)

def round_down_1(df):
    return df.round(1)

def round_down_2(df):
    return df.round(2)

def round_down_3(df):
    return df.round(3)

def round_up(df):
    return df.apply(np.ceil)


def resample_1ms(df):
    return df.resample('0.001S').mean()

def resample_1ms_ffill(df):
    return fill_nan_with_forward(resample_1ms(df))

def resample_1ms_bfill(df):
    return fill_nan_with_backward(resample_1ms(df))

def resample_1ms_linear(df):
    return fill_nan_with_linear(resample_1ms(df))


In [ ]:
plot_events(experiments_resampled, ['live_latency'], ylabel="Live latency (s)", legend_location="upper right", losses_to_plot=[0, 1 , 10, 20], start_time=start_time, end_time=end_time, data_modifier=[], correct_latency=True, show_experiment_dir=False, offset_time_on_mc=False)

In [ ]:
counter = 0
def remove_trend(df):
    global counter
    counter += 1
    if counter != 3:
        return df
    moving_avg = df['client_1_1 buffer_length'].rolling(100, min_periods=100).mean()
    ema = df['client_1_1 buffer_length'].ewm(alpha=0.0005, adjust=False).mean()
    df['client_1_1 buffer_length'] = df['client_1_1 buffer_length'] - ema + 0.8
    return df

In [ ]:
counter = 0

plot_events(experiments_resampled, ['buffer_length'], ylabel="Buffer length (s)", legend_location="upper right", losses_to_plot=[0, 1 , 10, 20], start_time=start_time, end_time=end_time, data_modifier=[remove_nan, resample_1ms], no_fec_only=True, offset_time_on_mc=False)

In [ ]:
plot_events(experiments, ['check_md5_time'], ylabel="Time to check md5 (ms)", legend_location="upper right", losses_to_plot=None, start_time=start_time, end_time=end_time, extra_event_filter=proxies_event_filter, data_modifier=[remove_nan, resample_1ms_linear], xlabel="Multicast time", offset_time_on_mc=True)

In [ ]:
plot_events(experiments_resampled, ['multicast_reception_time_before_deadline'], ylabel="Received before deadline (s)", legend_location="upper right", losses_to_plot=[0, 1, 10, 20], start_time=start_time, end_time=end_time, extra_event_filter=proxies_event_filter, data_modifier=[remove_nan, divide_data_by_1k], xlabel="Multicast time", offset_time_on_mc=True)

In [ ]:
plot_events(experiments_resampled, ['multicast_reception_time_after_deadline'], ylabel="Time since PR (s)", legend_location="upper right", losses_to_plot=[0, 1, 10, 20, 40, 80], start_time=start_time, end_time=end_time, extra_event_filter=proxies_event_filter, data_modifier=[remove_nan, divide_data_by_1k], xlabel="Multicast time", offset_time_on_mc=True, legend_columns=3, top_limit=1.16, bottom_limit=0, colors=['red', 'blue', 'green', 'orange', 'purple', 'brown'])  

In [ ]:
plot_events(experiments_resampled, ['multicast_reception_time_after_deadline'], ylabel="Time since PR (s)", legend_location="upper right", losses_to_plot=[0, 1, 10, 20, 40, 80], start_time=start_time, end_time=end_time, extra_event_filter=proxies_event_filter, data_modifier=[remove_nan, divide_data_by_1k], xlabel="Multicast time", offset_time_on_mc=True, show_video_title=False, show_loss=True, legend_columns=3, top_limit=1.16, bottom_limit=0, colors=['red', 'blue', 'green', 'orange', 'purple', 'brown'])  

In [ ]:
plot_events(experiments_resampled, ['playback_rate'], ylabel="Playback rate", legend_location="upper right", losses_to_plot=[0, 1, 10, 20, 80], start_time=start_time, end_time=end_time, extra_event_filter=None, data_modifier=[remove_nan], offset_time_on_mc=False)  

In [ ]:
plot_events(experiments_resampled, ['playing'], ylabel="Playback has started", legend_location="lower right", losses_to_plot=[0, 1, 10, 20, 80], start_time=start_time, end_time=end_time, extra_event_filter=None, data_modifier=[remove_nan, round_up], xlabel="Multicast time")  

In [ ]:
plot_events(experiments_resampled, ['latency_average'], ylabel="Average request latency (ms)", legend_location="lower right", losses_to_plot=[0, 1, 10, 20], start_time=start_time, end_time=end_time, extra_event_filter=None, data_modifier=[remove_nan, round_down_2, multiply_data_by_1k] )

In [ ]:
plot_events(experiments_resampled, ['ratio_average'], ylabel="Average ratio", legend_location="lower right", losses_to_plot=[0, 1, 10, 20, 80], start_time=start_time, end_time=end_time, extra_event_filter=None, data_modifier=[remove_nan])  

In [ ]:
plot_events(experiments_resampled, ['server_fetch_duration'], ylabel="Server latency (ms)", legend_location="upper right", losses_to_plot=[0, 1, 10, 20, 80], start_time=start_time, end_time=end_time, extra_event_filter=None, data_modifier=[remove_nan, divide_data_by_1k])  

In [ ]:
plot_events(experiments, ['symbols_received'], ylabel="Total symbols received", legend_location="lower right", losses_to_plot=[0, 1, 10, 20, 80], start_time=start_time, end_time=end_time, extra_event_filter=None, data_modifier=[remove_nan, resample_1ms_ffill], xlabel="Multicast time")  

In [ ]:
plot_events(experiments, ['fdt_received'], ylabel="Total FDTs received", legend_location="lower right", losses_to_plot=[0, 1, 10, 20, 80], start_time=start_time, end_time=end_time, extra_event_filter=None, data_modifier=[remove_nan, resample_1ms_ffill, round_down], xlabel="Multicast time")  

In [ ]:
plot_events(experiments, ['missing_symbols_gauge'], ylabel="Total symbols missing", legend_location="upper right", losses_to_plot=[0, 1, 10, 20, 80], start_time=start_time, end_time=end_time, extra_event_filter=None, data_modifier=[remove_nan], xlabel="Multicast time")  

In [ ]:
plot_events(experiments_resampled, ['multicast_files_received'], ylabel="Files received over MC", legend_location="lower right", losses_to_plot=[0, 1, 10, 20, 80], start_time=start_time, end_time=end_time, extra_event_filter=None, data_modifier=[remove_nan,round_down], xlabel="Multicast time")  

In [ ]:
plot_events(experiments, ['multicast_files_sent'], ylabel="Files transmitted over MC", legend_location="lower right", losses_to_plot=[0, 1, 10, 20, 80], start_time=start_time, end_time=end_time, extra_event_filter=None, data_modifier=[remove_nan, resample_1ms_ffill, set_nan_if_next_is_bigger, fill_nan_with_forward, round_down], xlabel="Multicast time")  

In [ ]:
plot_events(experiments, ['partial_requests'], ylabel="Recovery requess", legend_location="upper left", losses_to_plot=[0, 1, 10, 20], start_time=start_time, end_time=end_time, extra_event_filter=None, data_modifier=[remove_nan, resample_1ms_ffill, set_nan_if_next_is_bigger, fill_nan_with_forward, round_down], xlabel="Multicast time")  

In [ ]:
plot_events(experiments, ['multicast_transmission_time'], ylabel="MC transmission time", legend_location="upper right", losses_to_plot=[0, 1, 10, 20, 80], start_time=start_time, end_time=end_time, extra_event_filter=None, data_modifier=[remove_nan], xlabel="Multicast time")  

In [ ]:
plot_events(experiments, ['mtp'], ylabel="Average throughput (Mb/s)", legend_location="upper left", losses_to_plot=[0, 1, 10, 20], start_time=start_time, end_time=end_time, extra_event_filter=None, data_modifier=[remove_nan, divide_data_by_1k, resample_1ms_ffill, set_nan_if_next_is_bigger, fill_nan_with_forward, round_down_2])

In [ ]:
plot_events(experiments_resampled, ['files_fetched'], ylabel="Total files fetched", legend_location="upper left", losses_to_plot=[0, 1, 10, 20], start_time=start_time, end_time=end_time, extra_event_filter=None, data_modifier=[remove_nan, round_down], xlabel="Multicast time")

In [ ]:
plot_events(experiments_resampled, ['alcs_ignored'], ylabel="Total number of ALCs ignored", legend_location="lower right", losses_to_plot=[0, 1, 10, 20], start_time=start_time, end_time=end_time, extra_event_filter=None, data_modifier=[remove_nan, round_down], xlabel="Multicast time")

In [ ]:
plot_events(experiments, ['dropped_frames'], ylabel="Dropped frames", legend_location="lower right", losses_to_plot=[0, 1, 10, 20], start_time=start_time, end_time=end_time, extra_event_filter=None, data_modifier=[remove_nan, resample_1ms_ffill])

In [ ]:
plot_events(experiments_resampled, ['partial_processing_duration'], ylabel="Time to prepare recovery (ms)", legend_location="upper right", losses_to_plot=[0, 1, 10, 20, 80], start_time=start_time, end_time=end_time, extra_event_filter=None, data_modifier=[remove_nan,divide_data_by_1k, remove_columns_with_only_zeros], xlabel="Multicast time")

In [ ]:
plot_events(experiments_resampled, ['framerate'], ylabel="Frames per second", legend_location="lower right", losses_to_plot=[0, 1, 10, 20], start_time=start_time, end_time=end_time, extra_event_filter=None, data_modifier=[remove_nan,remove_columns_with_only_zeros])

In [ ]:
plot_events(experiments_resampled, ['file_hash_mismatches'], ylabel="Total hash mismatches", legend_location="lower right", losses_to_plot=[0, 1, 10, 20, 40, 80], start_time=start_time, end_time=end_time, extra_event_filter=None, data_modifier=[remove_nan, round_down], xlabel="Multicast time")